In [1]:
import torch
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import json

In [2]:
with open("../public_dataset/reference_images_part1.json") as f:
    ref_json = json.load(f)

In [ ]:
id_to_img = {}
df = pd.DataFrame()
dir_str = "../public_dataset/reference_images_part1"


for line in ref_json["images"]:
    filename = line["file_name"]
    img = Image.open(dir_str + "/" + filename)
    id_to_img[line["id"]] = img

for box in ref_json["annotations"]:
    box_coords = box["bbox"]
    # im1 = im.crop((left, top, right, bottom))
    # [lewy górny róg, szerokość, wysokość]
    box_coords = [box_coords[0], box_coords[1], box_coords[0] + box_coords[2], box_coords[1] + box_coords[3]]

    box["img"] = id_to_img[box["image_id"]].crop(box_coords)
    df = df.append(box, ignore_index=True)
    
                           
df.set_index("id")
df.head()

In [17]:
print(ref_json.keys())

dict_keys(['images', 'annotations', 'categories'])


In [5]:
print(ref_json["images"])

[{'id': 4, 'width': 1280, 'height': 720, 'file_name': 'nomagic_mulled_wine_3.png'}, {'id': 5, 'width': 1280, 'height': 720, 'file_name': 'nomagic_samsung_1.png'}, {'id': 6, 'width': 1280, 'height': 720, 'file_name': 'nomagic_brush_2.png'}, {'id': 7, 'width': 1280, 'height': 720, 'file_name': 'nomagic_cat_eye_1.png'}, {'id': 9, 'width': 1280, 'height': 720, 'file_name': 'nomagic_cat_eye_2.png'}, {'id': 11, 'width': 1280, 'height': 720, 'file_name': 'nomagic_boxers_2.png'}, {'id': 12, 'width': 1280, 'height': 720, 'file_name': 'nomagic_graphics_cards_4.png'}, {'id': 13, 'width': 1280, 'height': 720, 'file_name': 'nomagic_graphics_cards_3.png'}, {'id': 14, 'width': 1280, 'height': 720, 'file_name': 'nomagic_samsung_3.png'}, {'id': 15, 'width': 1280, 'height': 720, 'file_name': 'nomagic_light_bulb_1.png'}, {'id': 16, 'width': 1280, 'height': 720, 'file_name': 'nomagic_graphics_cards_2.png'}, {'id': 19, 'width': 1280, 'height': 720, 'file_name': 'nomagic_roller_3.png'}, {'id': 24, 'width': 

In [6]:
print(ref_json["categories"])

[{'id': 1, 'name': 'nomagic_brush'}, {'id': 2, 'name': 'nomagic_roller'}, {'id': 3, 'name': 'nomagic_digitec_cables'}, {'id': 4, 'name': 'nomagic_boxers'}, {'id': 5, 'name': 'nomagic_papiloty'}, {'id': 6, 'name': 'nomagic_maryna_experiment'}, {'id': 7, 'name': 'nomagic_toothpaste'}, {'id': 8, 'name': 'nomagic_mulled_wine'}, {'id': 9, 'name': 'nomagic_samsung'}, {'id': 10, 'name': 'nomagic_light_bulb'}, {'id': 11, 'name': 'nomagic_cat_eye'}, {'id': 12, 'name': 'nomagic_thinkpads'}, {'id': 13, 'name': 'nomagic_graphics_cards'}, {'id': 14, 'name': 'nomagic_wool_shaver'}, {'id': 15, 'name': 'nomagic_cherry'}, {'id': 16, 'name': 'nomagic_gargamel'}, {'id': 17, 'name': 'nomagic_foodie'}, {'id': 18, 'name': 'nomagic_dax_cream'}, {'id': 19, 'name': 'nomagic_cd'}, {'id': 20, 'name': 'nomagic_pet_vr'}, {'id': 21, 'name': 'nomagic_salt'}, {'id': 22, 'name': 'nomagic_advantech'}, {'id': 23, 'name': 'nomagic_familtea'}, {'id': 24, 'name': 'nomagic_joanna_grey'}, {'id': 25, 'name': 'nomagic_marshmal

In [7]:
print(ref_json["annotations"])

[{'id': 12, 'image_id': 4, 'bbox': [665, 448, 206, 174], 'category_id': 8}, {'id': 13, 'image_id': 4, 'bbox': [475, 186, 236, 186], 'category_id': 8}, {'id': 14, 'image_id': 4, 'bbox': [962, 76, 229, 217], 'category_id': 8}, {'id': 15, 'image_id': 4, 'bbox': [778, 250, 163, 135], 'category_id': 8}, {'id': 16, 'image_id': 4, 'bbox': [184, 377, 244, 194], 'category_id': 8}, {'id': 17, 'image_id': 4, 'bbox': [960, 422, 238, 228], 'category_id': 8}, {'id': 18, 'image_id': 4, 'bbox': [446, 437, 169, 155], 'category_id': 8}, {'id': 19, 'image_id': 4, 'bbox': [188, 136, 245, 183], 'category_id': 8}, {'id': 20, 'image_id': 5, 'bbox': [718, 152, 331, 187], 'category_id': 9}, {'id': 21, 'image_id': 5, 'bbox': [215, 116, 146, 262], 'category_id': 9}, {'id': 22, 'image_id': 5, 'bbox': [206, 409, 272, 236], 'category_id': 9}, {'id': 23, 'image_id': 5, 'bbox': [811, 265, 306, 233], 'category_id': 9}, {'id': 24, 'image_id': 5, 'bbox': [452, 119, 253, 244], 'category_id': 9}, {'id': 25, 'image_id': 5,